## Ejercicio 

### Haciendo proyecto de IMDB en Pytorch

In [ ]:
import torch                     # for all things PyTorch
import torch.nn as nn            # for torch.nn.Module, the parent object for PyTorch models
import torch.nn.functional as F  # for the activation function

# Get APIs for optimizer, scheduler and datasets
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from torchvision.transforms import ToTensor

In [2]:
from logging import logProcesses
import os, shutil
from platform import python_branch
from syslog import LOG_SYSLOG

import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.layers import Dropout
from keras.datasets import imdb

print ("TF Version   ", tf.__version__)
print ("TF Path      ", tf.__path__[0])
print("Keras version ", keras.__version__)
print("numpy version ", np.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-03-02 00:40:26.468378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740876026.518140    1783 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740876026.531637    1783 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 00:40:26.637218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TF Version    2.18.0
TF Path       /usr/local/lib/python3.11/site-packages/keras/api/_v2
Keras version  3.8.0
numpy version  2.0.2
Num GPUs Available:  1


In [3]:
import os
import csv

# get the current working directory
current_working_directory = os.getcwd()
# print output to the console
print(current_working_directory)

/home/jupyter


In [4]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)).astype('float32')
    for i, sequence in enumerate (sequences):
        results[i, sequence] = 1
    return results

def print_review( item ) :
  for i in item :
    print (i)

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
# Get the data from tf/keras. Maybe I need to test the local data to ensure it is a bona fide version 
imdb = tf.keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

print (train_data[0])


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
print (train_labels[10:])
print (train_data.shape)
print (train_labels.shape)
print (test_data.shape)
print (test_labels.shape)

[1 0 0 ... 0 1 0]
(25000,)
(25000,)
(25000,)
(25000,)


In [8]:
# with open('train_data.txt', 'w') as f:
#     csv.writer(f, delimiter=' ').writerows(train_data)

# with open('train_labels.txt', 'w') as f:
#     csv.writer(f, delimiter=' ').writerows(train_labels)

In [7]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = torch.tensor(np.asarray(train_labels)).to(torch.long)#.astype('long')
y_test  = torch.tensor(np.asarray(test_labels)).to(torch.long)#.astype('float32')



In [9]:
print (x_train[0])
print (type(x_train))
print (x_train.shape)
print (train_labels.shape)


[0. 1. 1. ... 0. 0. 0.]
<class 'numpy.ndarray'>
(25000, 10000)
(25000,)


In [10]:
# DATA_train = [[torch.tensor(d.tolist()).to(torch.int), l.item()] for d, l in zip(x_train, y_train)]
DATA_train = [[torch.tensor(d).to(torch.float), l.item()] for d, l in zip(x_train, y_train)]

# DATA_train = [[torch.tensor(d), l.item()] for d, l in zip(x_train, y_train)]
print (DATA_train[0])

[tensor([0., 1., 1.,  ..., 0., 0., 0.]), 1]


In [11]:
DATA_test = [[torch.tensor(d).to(torch.float), l.item()] for d, l in zip(x_test, y_test)]
print (DATA_test[0])

[tensor([0., 1., 1.,  ..., 0., 0., 0.]), 0]


In [12]:
# Define a NN model with dense layers (which Pytorch calls "Linear").
# The model can be used to process the MNIST-digits data 

# Determine what device (GPU, CPU, MPS) is available for training

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# # create a model witl 16 units, compiule, train, plot
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(10000,)))
# model.add(tf.keras.layers.Dense(16, activation='relu'))
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])


# Define Dense Model
class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10000,16),
            nn.ReLU(),
            nn.Linear(16,16),
            nn.ReLU(),
            nn.Linear(16,2),
            # nn.ReLU(),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    # def forward(self, x):
    #     x = self.flatten(x)
    #     logits = self.linear_relu_stack(x)
    #     return logits
dense_model = DenseNet().to(device)
print(dense_model)
print("Numero de parámetros=",count_params(dense_model))

Using cpu device
DenseNet(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10000, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=2, bias=True)
    (5): Sigmoid()
  )
)
Numero de parámetros= 160322


In [ ]:
# input_dim=100
# embedding_dim=128
# hidden_dim=258
# output_dim=2

# # Define Dense Model
# class DenseNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.embedding=nn.Embedding(100,128)
#         # self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(10000,16),
#             nn.ReLU(),
#             nn.Linear(16,2),
#             # nn.ReLU(),
#             # nn.Linear(16,2),
#             # nn.ReLU(),
#             # nn.Sigmoid(),
#         )

#     def forward(self, x):
#         embedding=self.embedding(x)
#         # x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

#     # def forward(self, x):
#     #     x = self.flatten(x)
#     #     logits = self.linear_relu_stack(x)
#     #     return logits
# dense_model = DenseNet().to(device)
# print(dense_model)
# print("Numero de parámetros=",count_params(dense_model))

In [13]:
# x_val = x_train[:10000]
# partial_x_train = x_train[10000:]

# y_val = y_train[:10000]
# partial_y_train = y_train[10000:]

# history = model_8.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

# history_dict = history.history
# loss_values = history_dict['loss']
# val_loss_values = history_dict['val_loss']
# epochs = range(1, len(loss_values) + 1)


# Create data loaders
# combined2
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=True)
train_loader = torch.utils.data.DataLoader(DATA_train, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(DATA_test, batch_size=256, shuffle=False)


In [14]:
learning_rate = 0.005
# Instantiate the model, optimizer, and loss function
model = DenseNet()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [15]:
model.__init__()

In [ ]:


# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# Define the training loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Define the testing loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    i=0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            # if(i % 500 == 0): print(output)
            test_loss += criterion(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            i+=1

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                                    test_loss,      correct, len(test_loader.dataset),
                                                                100. * correct / len(test_loader.dataset)))


In [17]:

nEpochs = 10

# Train the model
for epoch in range(1, nEpochs + 1):  # Train for 10 epochs
    train(epoch)
    test()

Train Epoch: 1 [0/25000 (0%)]	Loss: 0.692196

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 2 [0/25000 (0%)]	Loss: 0.693583

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 3 [0/25000 (0%)]	Loss: 0.694071

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 4 [0/25000 (0%)]	Loss: 0.692780

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 5 [0/25000 (0%)]	Loss: 0.693540

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 6 [0/25000 (0%)]	Loss: 0.694070

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 7 [0/25000 (0%)]	Loss: 0.691958

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 8 [0/25000 (0%)]	Loss: 0.692355

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 9 [0/25000 (0%)]	Loss: 0.693037

Test set: Average loss: 0.0027, Accuracy: 12500/25000 (50%)

Train Epoch: 10 [0/25000 (0%)]	Loss: 